<a href="https://colab.research.google.com/github/cbotengine/cbot/blob/main/main_ori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pythainlp -q
! pip install epitran -q
! pip install sklearn_crfsuite -q
! pip install tensorflow deepcut -q
! pip install attacut -q
! pip install phunspell -q
! pip install symspellpy -q
! pip install newmm-tokenizer

! pip install pandas -q
! pip install nltk -q
! pip install xlrd -q
! pip install keras-metrics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 KB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 12.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel 

In [ ]:
import pythainlp

from pythainlp import Tokenizer, word_tokenize
from pythainlp.util import dict_trie
from pythainlp.corpus.common import thai_words

def word_cut(msg):
    
    words_list = set(thai_words())
    match_word = dict_trie(dict_source=words_list)
    custom_tokenizer = Tokenizer(custom_dict=match_word, engine='newmm')
    word_cut = custom_tokenizer.word_tokenize(msg)

    return word_cut

In [ ]:
import pythainlp
import phunspell
from pythainlp import correct

def word_spell(msg):

    word_spell = ''
    
    for i,word in enumerate(msg):
        if word != ' ' and word != '(' and word != ')':
            word_spell = word_spell + correct(word, engine='phunspell')
        else:
            word_spell = word_spell + word

    return word_spell

In [ ]:
import pythainlp
from pythainlp.tokenize import Tokenizer, word_tokenize
from pythainlp.corpus import thai_words, thai_stopwords
from pythainlp.util import dict_trie

def word_stop(msg):

    words_list = set(thai_words())
    match_word = dict_trie(dict_source=words_list)
    custom_tokenizer = Tokenizer(custom_dict=match_word, engine='newmm')
    list_word = custom_tokenizer.word_tokenize(msg)

    list_stop = list(thai_stopwords())   
    word_stop = [i for i in list_word if i not in list_stop]

    return word_stop

In [ ]:
###TEST
text = "ไม่ค่อยเข้าใจช่วยอธิบายหน่อย"
msg_word_cut = word_cut(text)
msg_word_spell = word_spell(msg_word_cut)
msg_word_stop = word_stop(msg_word_spell)

print(msg_word_cut)
print(msg_word_spell)
print(msg_word_stop)

['ไม่ค่อย', 'เข้าใจ', 'ช่วย', 'อธิบาย', 'หน่อย']
ไม่ ค่อยเข้าใจช่วยอธิบายหน่อย
[' ', 'อธิบาย']


In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
import tensorflow as tf 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random


#import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')

words=[]
classes = []
documents = []
ignore_words = ['?', '!']

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
intents = {"intents": [
{"tag": "H1",
 "patterns":["สวัสดี","ดี","ทัก","สวัสดีจ้า","ดีจ้า"],
 "responses": ["ชิปป์สไมล์ ยินดีต้อนรับลูกค้าทุกท่านค่ะ ต้องการให้ทางร้านชิปป์สไมล์ช่วยเหลือเรื่องอะไรสามารถสอบถามได้เลยค่ะ"]},
{"tag": "H2",
 "patterns":["ขอบคุณ","ขอบคุณครับ","ขอบคุณค่ะ"],
 "responses": ["ร้าน ชิปป์สไมล์ ยินดีให้บริการ"]},
{"tag": "S1",
 "patterns":["ร้านปิดกี่โมง","ร้านเปิดกี่โมง","ร้านปิดตอนไหน","ร้านเปิดตอนไหน","ร้านเปิดแล้วหรือยัง","ไม่ทราบพรุ่งนี้ร้านปิดมั้ยคะ"], 
 "responses": ["ร้านเปิดให้บริการทุกวัน วันจันทร์ - วันเสาร์ เวลา 8.30 - 18.30 น. วันอาทิตย์ เวลา 9.00 - 17.00 น."]},
{"tag": "S2",
 "patterns":["มีส่งไปต่างประเทศไหม","ต่างประเทศ","ส่งสินค้าไปต่างประเทศได้ไหม","ส่งไปต่างประเทศมั่ย","ส่งไปต่างประเทศคิดราคายังไง"], 
 "responses": ["ทางร้านไม่มีขนส่งไปต่างประเทศค่ะ"]},
{"tag": "S3",
 "patterns":["ที่ร้านมีซองกระดาษสีน้ำตาลขนาด 11.5x16.5 ไหม"], 
 "responses": ["มีค่ะ กระดาษ A3 แบบขนาด 11x17"]},
{"tag": "S4",
 "patterns":["ส่งพัสดุได้ถึงกี่โมง","รอบการส่งพัสดุ","ส่งได้ถึงกี่โมง","รถมาเอาของกี่โมง","วันนี้Flash มารับของกี่โมง","ที่ร้านตัดรอบกี่โมง"], 
 "responses": ["ทางร้านตัดรอบการจัดส่ง เวลา 14.00 น. ค่ะ"]},
{"tag": "S5",
 "patterns":["ส่งพื้นที่ห่างไกลบวกเพิ่มไหม","พื้นที่ห่างไกล","ส่งไปพื้นที่ห่างไกลได้ไหม"], 
 "responses": ["ทางร้านรองรับการจัดส่งพื้นที่ห่างไกล ฟรีค่าจัดส่งค่ะ"]},
{"tag": "S6",
 "patterns":["สถานะการจัดส่งพัสดุ","สถานะพัสดุ","สถานะการส่งพัสดุ","สอบถามเลขพัสดุ","สอบถามสถานะการจัดส่งพัสดุค่ะ","พอดียังไม่ได้รับสินค้า"], 
 "responses": ["ขอเลขบัตรประชาชนเพื่อทำการตรวจสอบเลขพัสดุค่ะ"]},
{"tag": "S7",
 "patterns":["มีบริการปลายทางไหม","มีบริการ COD ไหม"], 
 "responses": ["ทางร้านมีบริการเก็บเงินปลายทางของ Flash , Kerry , และ Ninja Van ค่ะ"]},
{"tag": "S8",
 "patterns":["สอบถามหน่อย","สอบถาม","ถามหน่อย"], 
 "responses": ["ต้องการให้ทางร้านชิปป์สไมล์ช่วยเหลือเรื่องอะไร สามารถสอบถามได้เลยค่ะ"]},
{"tag": "S9",
 "patterns":["มีขนส่งอะไรบ้าง","ที่ร้านมีขนส่ง j t มั้ยค่ะ"], 
 "responses": ["EMS ไปรษณีย์ไทย , Flash Express , Kerry Express , J&T , DHL (ภายในประเทศ) "]},
{"tag": "S10",
 "patterns":["มีพัสดุจัดส่ง แบบสมมุติส่งวันนี้ถึงพรุ่งนี้บ้างไหม"], 
 "responses": ["แนะนำเป็น Kerry Express ส่วนมากจะถึงในวันถัดไป แต่ต้องส่งก่อนตัดรอบ เวลา 14.00 น."]},
{"tag": "S11",
 "patterns":["ของ flash ใช้เวลาประมาณกี่วันของถึง","ด่วนพิเศษ ใช้เวลาประมาณกี่วันถึงปลายทาง","ที่ร้านมีส่งด่วนไหม"], 
 "responses": ["ใช้เวลาการจัดส่งไม่เกิน 3 วันค่ะ"]},
{"tag": "S12",
 "patterns":["บวกค่าเสี่ยงภัยกี่บาท","บวกค่าเสี่ยงภัยราคาเท่าไหร่","ราคาค่าเสี่ยงภัย","ค่าเสี่ยงภัยคิดยังไง","เสี่ยงภัย"], 
 "responses": ["ค่าเสี่ยงภัยบวก 50 บาทค่ะ"]},
{"tag": "S13",
 "patterns":["ค่าเสี่ยงภัยของไปรษณีย์ไทย"], 
 "responses": ["ไปรษณีไทยไม่มีค่าเสี่ยงภัย"]},
{"tag": "S14",
 "patterns":["สินค้ามีปัญหาขอรับของคืนได้ไหม"], 
 "responses": ["หากพัสดุยังไม่ได้จัดส่งสามารถมารับสินค้าคืนได้เลยค่ะ พร้อมนำบัตรประจำตัวประชาชนใช้เพื่อการยกเลิก"]},
{"tag": "S15",
 "patterns":["ทางร้านรับคนละครึ่งไหม"], 
 "responses": ["ทางร้านไม่เข้าร่วมโครงการคนละครึ่งค่ะ"]},
{"tag": "S16",
 "patterns":["หากเป็นออเดอร์จากช้อปปี้ คือจะส่งฟรีทุกรายการเลยใช่ไหมคะ หรือจำกัดน้ำหนัก"], 
 "responses": ["ออเดอร์จาก Shopee ส่งฟรีทุกรายการคะ"]},
{"tag": "S17",
 "patterns":["วิธีเช็คยอด COD เช็คยังไง"], 
 "responses": ["ยอด COD จะมีการแจ้งผ่านทางอีเมลล์ค่ะ"]},
{"tag": "S18",
 "patterns":["ทางร้านสามารถปริ้นได้ไหม"], 
 "responses": ["สามารถให้ทางร้านปริ้นเอกสารให้ได้ค่ะ"]},
{"tag": "S19",
 "patterns":["เช็คสถานะในระบบได้เมื่อไหร่","พี่เช็คเลขพัสดุ แต่มันบอกไม่พบเลขพัสดุ"], 
 "responses": ["สามารถเช็คสถานะพัสดุได้ เวลา 11.30 น. ในวันถัดไปหลังจากจัดส่งสินค้าค่ะ"]},
{"tag": "S20",
 "patterns":["ช่วงสงกรานต์ใช้เวลาส่งกี่วันค่ะ"], 
 "responses": ["ช่วงสงการนต์ ทุกขนส่งจะล่าช้า 1-2 วันค่ะ"]},
{"tag": "S21",
 "patterns":["สินค้าชำรุด สามารถเครมได้ไหม","แปะระวังแตกไว้ แต่พอของถึงพังยับเยินเลย"], 
 "responses": ["รบกวนขอเลขพัสดุและภาพประกอบ ทางร้านจะรวบรวมและยื่นเครมสินค้าให้ค่ะ"]},
{"tag": "S22",
 "patterns":["ที่ร้านรับถ่ายเอกสารแผ่นละเท่าไหร่"], 
 "responses": ["ถ่ายเอกสารขาวดำแผ่นละ 1 บาทค่ะ"]},
{"tag": "S23",
 "patterns":["ทางร้านส่งไปรษณีย์ได้ไหม"], 
 "responses": ["ไปรษณีย์สามารถส่งได้ EMS และ Ecopost ค่ะ"]},
{"tag": "S24",
 "patterns":["ถ้าตีกลับของต้องทำยังไงบ้าง"], 
 "responses": ["ลูกค้าสามารถปฏิเสธการรับของได้เลยค่ะ"]},
{"tag": "S25",
 "patterns":["ค่าส่งเท่าไหร่"], 
 "responses": ["ถ้าน้ำหนักไม่เกิน 1Kg. ราคา 20 บาทค่ะ"]},
{"tag": "S26",
 "patterns":["ส่งไปรษณีย์ราคาเท่าไหร่","ส่ง EMS ราคากี่บาท"], 
 "responses": ["ส่ง EMS ราคา 37 บาทค่ะ"]},
{"tag": "S27",
 "patterns":["มีส่งแบบแช่เย็นไหม"], 
 "responses": ["ทางร้านยังไม่รับส่งสินค้าแบบแช่เย็นค่ะ"]},
{"tag": "S28",
 "patterns":["มีบับเบิ้ลให้ห่อไหม"], 
 "responses": ["ทางร้านมีบับเบิ้ลขายค่ะ หากซื้อกล่องกับทางร้านสามารถใช้บับเบิ้ลฟรีได้เลยค่ะ"]}
]}


In [ ]:
list_word = []
list_class = []

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

        list_word.append(intent['patterns'])
        list_class.append(intent['tag'])

#documents
#classes

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

70 documents
30 classes ['H1', 'H2', 'S1', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S2', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']
89 unique lemmatized words ['11.5x16.5', 'cod', 'em', 'flash', 'j', 't', 'ของ', 'ขอบคุณ', 'ขอบคุณครับ', 'ขอบคุณค่ะ', 'คือจะส่งฟรีทุกรายการเลยใช่ไหมคะ', 'ค่าส่งเท่าไหร่', 'ค่าเสี่ยงภัยของไปรษณีย์ไทย', 'ค่าเสี่ยงภัยคิดยังไง', 'ช่วงสงกรานต์ใช้เวลาส่งกี่วันค่ะ', 'ดี', 'ดีจ้า', 'ด่วนพิเศษ', 'ต่างประเทศ', 'ถามหน่อย', 'ถ้าตีกลับของต้องทำยังไงบ้าง', 'ทัก', 'ทางร้านรับคนละครึ่งไหม', 'ทางร้านสามารถปริ้นได้ไหม', 'ทางร้านส่งไปรษณีย์ได้ไหม', 'ที่ร้านตัดรอบกี่โมง', 'ที่ร้านมีขนส่ง', 'ที่ร้านมีซองกระดาษสีน้ำตาลขนาด', 'ที่ร้านมีส่งด่วนไหม', 'ที่ร้านรับถ่ายเอกสารแผ่นละเท่าไหร่', 'บวกค่าเสี่ยงภัยกี่บาท', 'บวกค่าเสี่ยงภัยราคาเท่าไหร่', 'พอดียังไม่ได้รับสินค้า', 'พี่เช็คเลขพัสดุ', 'พื้นที่ห่างไกล', 'มั้ยค่ะ', 'มารับของกี่โมง', 'มีขนส่งอะไรบ้าง', 'มีบริการ', 'มีบริการปลายทางไหม', 'มีบับ

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
#model = load_model('chatbot_model.h5')
model = load_model('chatbot_model.h5')
import json
import random
#intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

#model.save('chatbot_model.h5', hist)
model.save('model.h5', hist)

#print("model created")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
14/14 [==============================] - 1s 2ms/step - loss: 3.4364 - accuracy: 0.0143
Epoch 2/200
14/14 [==============================] - 0s 2ms/step - loss: 3.3768 - accuracy: 0.0571
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 3.3199 - accuracy: 0.1143
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 3.2934 - accuracy: 0.0714
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 3.2368 - accuracy: 0.0857
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 3.1984 - accuracy: 0.1000
Epoch 7/200
14/14 [==============================] - 0s 2ms/step - loss: 3.2217 - accuracy: 0.1143
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 3.1531 - accuracy: 0.1286
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 3.0945 - accuracy: 0.1286
Epoch 10/200
14/14 [==============================] - 0s 2ms/step - loss: 3.1036 - accuracy: 0.1429
Epoch 11/

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)  #JRP ตัดคำ
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False) #JRP ตัดคำ
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.1
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:   
        #if r[1] > 0.5:
          #return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
          return_list.append({"intent": classes[r[0]]})
        #else:
        #  return_list.append({"intent": "H1", "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    msg_word_cut = word_cut(text)
    msg_word_spell = word_spell(msg_word_cut)
    msg_word_stop = word_stop(msg_word_spell)
    msg_ints = predict_class(msg_word_spell, model)
    #msg_ints = predict_class(msg_word_stop, model)
    res = getResponse(msg_ints, intents)


    print ("==== Result ====")
    print ("+++ msg_word_cut = ",msg_word_cut)
    print ("+++ msg_word_spell = ",msg_word_spell) 
    print ("+++ msg_word_stop = ",msg_word_stop) 
   #print ("+++ msg_ints = ",msg_ints) 
    print ("         ")
    print ("==== Answer ====")
    #t = timeit.Timer('char in text', setup='text = "sample string"; char = "g"')
    #t.timeit()
    return res

In [ ]:
# Let's test the model now:
chatbot_response("ไปรษณีย์")

1/1 [==============================] - 0s 21ms/step
==== Result ====
+++ msg_word_cut =  ['ไปรษณีย์']
+++ msg_word_spell =  ไปรษณีย์
+++ msg_word_stop =  ['ไปรษณีย์']
         
==== Answer ====


'ทางร้านไม่มีขนส่งไปต่างประเทศค่ะ'

In [ ]:
chatbot_response("ค่าเสี่ยงภัยของไปรษณีย์ไทย")

1/1 [==============================] - 0s 29ms/step
==== Result ====
+++ msg_word_cut =  ['ค่า', 'เสี่ยงภัย', 'ของ', 'ไปรษณีย์', 'ไทย']
+++ msg_word_spell =  ค่าเสี่ยงภัยของไปรษณีย์ไทย
+++ msg_word_stop =  ['ค่า', 'เสี่ยงภัย', 'ไปรษณีย์', 'ไทย']
         
==== Answer ====


'ไปรษณีไทยไม่มีค่าเสี่ยงภัย'

In [ ]:
chatbot_response("รอบการส่งพัสดุ")

1/1 [==============================] - 0s 19ms/step
==== Result ====
+++ msg_word_cut =  ['รอบ', 'การ', 'ส่ง', 'พัสดุ']
+++ msg_word_spell =  รอบการส่งพัสดุ
+++ msg_word_stop =  ['รอบ', 'พัสดุ']
         
==== Answer ====


'ทางร้านตัดรอบการจัดส่ง เวลา 14.00 น. ค่ะ'

In [ ]:
chatbot_response("มีขนส่งอะไรบ้าง")

1/1 [==============================] - 0s 20ms/step
==== Result ====
+++ msg_word_cut =  ['มี', 'ขนส่ง', 'อะไร', 'บ้าง']
+++ msg_word_spell =  มีขนส่งอะไรบ้าง
+++ msg_word_stop =  ['ขนส่ง']
         
==== Answer ====


'EMS ไปรษณีย์ไทย , Flash Express , Kerry Express , J&T , DHL (ภายในประเทศ) '

In [ ]:
chatbot_response("มีบริการปลายทางไหม")

1/1 [==============================] - 0s 20ms/step
==== Result ====
+++ msg_word_cut =  ['มี', 'บริการ', 'ปลายทาง', 'ไหม']
+++ msg_word_spell =  มีบริการปลายทางไหม
+++ msg_word_stop =  ['บริการ', 'ปลายทาง', 'ไหม']
         
==== Answer ====


'ทางร้านมีบริการเก็บเงินปลายทางของ Flash , Kerry , และ Ninja Van ค่ะ'

In [ ]:
chatbot_response("ทางร้สนส่งไปรษณีย์ได้ไหม")

1/1 [==============================] - 0s 19ms/step
==== Result ====
+++ msg_word_cut =  ['ทาง', 'ร้สน', 'ส่ง', 'ไปรษณีย์', 'ได้', 'ไหม']
+++ msg_word_spell =  ทางร้านส่งไปรษณีย์ได้ไหม
+++ msg_word_stop =  ['ร้าน', 'ไปรษณีย์', 'ไหม']
         
==== Answer ====


'ไปรษณีย์สามารถส่งได้ EMS และ Ecopost ค่ะ'

In [ ]:
accr = model.evaluate(train_x,train_y, verbose=1) #ทดสอบความแม่นยำของโมเดล กับ Test set ที่แยกไว้ตั้งแต่แรกอีกที \n",

3/3 [==============================] - 0s 4ms/step - loss: 6.1529e-04 - accuracy: 1.0000
